In [5]:
import numpy as np
import math
class Node:
    def __init__(self, A, B, C,fuel_cost, pMin, pMax, on_cost , off_cost , label):

        self.A=A
        self.B = B
        self.C = C
        self.fuel_cost = fuel_cost
        self.pMax= pMax
        self.pMin = pMin
        self.on_cost = on_cost
        self.off_cost = off_cost
        self.flapc = self.FLAPC()
        self.label = label
        self.active_power = 0
        self.active_cost = 0
    def get_active_cost(self,power):
        return (self.fuel_cost * (self.A + self.B * power+ self.C* power * power) + self.on_cost + self.off_cost)
    def FLAPC (self):
        # FLAPC = fuel_cost * (A+BPmax+cPmax^2)/Pmax
        return self.get_active_cost(self.pMax)/self.pMax
    def reset(self):
        self.active_power = 0
        self.active_cost = 0

In [2]:
def find_feasible_combination(nodes , load):
    nodes.sort(key=lambda node:node.flapc)
    feasible_nodes = []
    new_load  = load
    for node in nodes:
        if new_load <= 0:
            return feasible_nodes
        new_load -= node.pMax
        feasible_nodes.append(node)
    return feasible_nodes

def get_lambda(nodes , load):
    num =0
    denom = 0
    if len(nodes) == 0:
        return 0
    for node in nodes:
        num += node.B/(2.0 *node.C)
        denom += 1.0/(2.0*node.C)
        #print(num, denom)
    return ((load + num) /denom)

In [3]:
def get_power(nodes , load):
    num = 0
    denom = 0
    #for load in loads:
    feasible_nodes = find_feasible_combination(nodes , load)
    if(len(feasible_nodes)):
        lambda1 = get_lambda(feasible_nodes , load)
        newLoad = load
        for node in feasible_nodes:
            if newLoad <= 0.0:
                break
            power = (lambda1 - node.B)/(2.0*node.C)
            if(power > node.pMax):
                power = node.pMax
                node.active_power= power
                lambda1 = get_lambda( [node for node in feasible_nodes if node.active_power <= 0] ,
                                     (newLoad - power))
            newLoad -= power
            node.active_cost = node.get_active_cost(power)
            if node.active_power<= 0:
                node.active_power= power
        print("\n Load :" , load , 
          "\n nodes " ,[node.label for node in feasible_nodes] ," are used.", 
          "\n node active power :"  , [node.active_power for node in feasible_nodes] ,
          "\n Total generated power:", sum([node.active_power for node in feasible_nodes]),
          "\n node active cost:", [node.active_cost for node in feasible_nodes],
          "\n Total cost:",sum([node.get_active_cost(node.active_power) for node in feasible_nodes]))
        for node in feasible_nodes:
            node.reset()

In [4]:
node1 = Node(A=30 , B=25 ,C=100,fuel_cost = 1.1, pMin=150 , pMax= 600 ,on_cost=1, off_cost=1, label = "node1")
node2 = Node(A=50 , B=10 ,C=80,fuel_cost = 1.0, pMin=100 , pMax= 400 ,on_cost=1, off_cost=1, label = "node2")
node3 = Node(A=20 , B=20 ,C=70,fuel_cost = 1.2, pMin=50 , pMax= 200 ,on_cost=1, off_cost=1, label = "node3")

loads =[[700.0 , 900.0],[200.0 , 500.0]]
nodes = [node1 , node2 , node3]

for load in loads:
    get_power(nodes ,sum(load))


 Load : 1600.0 
 nodes  ['node3', 'node2', 'node1']  are used. 
 node active power : [200, 400, 600] 
 Total generated power: 1200 
 node active cost: [3364826.0, 12804052.0, 39616535.0] 
 Total cost: 55785413.0

 Load : 700.0 
 nodes  ['node3', 'node2', 'node1']  are used. 
 node active power : [200, 277.81944444444446, 222.18055555555554] 
 Total generated power: 700.0 
 node active cost: [3364826.0, 6177521.691358024, 5436206.8846450625] 
 Total cost: 14978554.576003086
